In [ ]:
# @title Install libraries
# @markdown This cell will take a while because you have to download multiple libraries
 
!pip install transformers scipy ftfy "ipywidgets>=7,<8"
!git clone https://github.com/huggingface/diffusers.git
!pip install git+https://github.com/huggingface/diffusers.git
cd diffusers

In [ ]:
# @title Login in Hugging Face
# @markdown Login in hugginggace and take in account setting access token https://huggingface.co/ .

# @markdown And in this link https://huggingface.co/CompVis/stable-diffusion-v1-4 you must accept the agreement on the use of this model (check the box).
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# @title Import libraries and define things
import os
import torch
import shutil
from PIL import Image
from torch import autocast
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from diffusers import StableDiffusionPipeline
from examples.inference.image_to_image import StableDiffusionImg2ImgPipeline, preprocess
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True).to("cuda")
pipeimg = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True).to("cuda")

In [ ]:
#@title (Run after from here to make changes)
# @markdown Your prompt and size image
PROMPT = "Test" #@param {type:"string"}
# @markdown i recommend use small size of image because Free Colab don't have more GPU
HEIGHT = 512 #@param {type:"integer"}
WIDTH = 512 #@param {type:"integer"}
# @markdown Number of example
EXAMPLES = 1 #@param {type:"integer"}
# @markdown Number of step (recommend 50-100)
STEPS = 50 #@param {type:"integer"}
GUIDANCE_SCALE = 7.5 #@param {type:"number"}
# @markdown for image2image generating
STRENGTH = 0.75 #@param {type:"number"}
INIT_IMAGE = "" #@param {type:"string"}
#SAVE_DIR = ""
try:
  if not PROMPT.strip(): #если строка промпта пустая
    print("Please write your prompt")
    pass
  else: #если не пустая
    for i in range(EXAMPLES): #кол-во примеров
      if not INIT_IMAGE.strip(): #если изображение пусто
        try:
          shutil.rmtree(f'{PROMPT}') #Удаляет папку если она есть
        except FileNotFoundError:
          pass
        os.mkdir(f'{PROMPT}')
        with autocast("cuda"):
          plt.figure(figsize=(8,8))
          image = pipe(PROMPT, num_inference_steps=STEPS, height=HEIGHT, width=WIDTH, guidance_scale=GUIDANCE_SCALE)["sample"][0]
          image.save(f'{PROMPT}/test{i}.png')
          imgi = mpimg.imread(f'{PROMPT}/test{i}.png')
          imgplot = plt.imshow(imgi)
          plt.axis('off')
          plt.show()
      else: #если изображение есть
        init_image = Image.open(INIT_IMAGE).convert("RGB")
        init_image = init_image.resize((WIDTH, HEIGHT))
        init_image = preprocess(init_image)
        prompt_save = "init image__" + prompt
        try:
          shutil.rmtree(f'{prompt_save}') 
        except FileNotFoundError:
          pass
        os.mkdir(f'{prompt_save}')
        with autocast("cuda"):
          plt.figure(figsize=(8,8))
          image = pipeimg(PROMPT, init_image=init_image, strength=STRENGTH, num_inference_steps=STEPS, guidance_scale=GUIDANCE_SCALE)["sample"][0]
          image.save(f'{prompt_save}/test{i}.png')
          imgi = mpimg.imread(f'{prompt_save}/test{i}.png')
          imgplot = plt.imshow(imgi)
          plt.axis('off')
          plt.show()
except:
  pass